# Moment tensor inversion of perforation shots

This concerns perforation shots in the monitor well using near positive offsets (stimulated part of the reservoir).
The aim here is to use the MT as a garbage collector to fit as best as possible the near positive offsets. Therefore, it is not expected to derive too much interpretation from the results. The offset range used in the inversions is [150 - 200 m].

In [ ]:
import sys

pwd = !echo ${PWD}
sys.path.append(pwd[0]+"/../../../code/local/bin")

In [ ]:
import numpy as np
import pandas as pd
from scipy import signal
from scipy import ndimage
import math
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.ticker import FormatStrFormatter

import seppy
import os

datapath=pwd[0]+"/../dat/"
figpath=pwd[0]+"/../fig/"

In [ ]:
# utility functions

def plotseis(d, p=0.9, valmin=1, valmax=-1, tmin=0, tmax=0.3, xmin=-500, xmax=500, grid=True, colorbar=False, curve=np.array([]), linestyle='--'):
    """Plot 2D seismic image"""
 
    vmin=np.min(d) / p
    vmax=p*np.max(d)
    
    if p<0:
        vmax=-p*np.amax(d)
        vmin=-vmax
        
    if valmin<valmax:
        vmin=valmin
        vmax=valmax
        
    nx=d.shape[0]
                
    fig=plt.figure(figsize=(6, 4),dpi=100)
    plt.imshow(np.transpose(d),interpolation='sinc',aspect="auto",extent=[xmin,xmax,tmax,tmin],cmap='Greys',vmin=vmin,vmax=vmax)
    plt.xlabel('Offset (m)')
    plt.ylabel('Time (s)')
    if grid==True:
        plt.grid(color='w', linestyle='-', linewidth=0.2)
    if colorbar==True:
        plt.colorbar()
    if curve.size>0:
        plt.plot(np.linspace(xmin,xmax,nx),curve,linestyle)
    plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
    
    plt.show()

def wavelet(nt=101, dt=0.002, t0=0, sig1=0.003, sig2=0.02, derivative=True):
    """Build a time function that is the generalized Hubbert's pimple (or its derivative)"""
    
    it0=int(nt/2)
    t=(np.linspace(0,nt-1,nt)-it0)*dt
    a=(sig2/sig1)**(-sig2/(sig1+sig2)) + (sig2/sig1)**(sig1/(sig1+sig2))
    a=a*a
    if derivative==True:
        w=-a*(-np.exp((t0-t)/sig1)/sig1 + np.exp((t-t0)/sig2)/sig2)/(np.exp((t0-t)/sig1) + np.exp((t-t0)/sig2))**3
    else:
        w=0.5*a/(np.exp((t0-t)/sig1) + np.exp((t-t0)/sig2))**2
    
    return w

In [ ]:
# read data, Green's functions, mask, attributes

# no plug between stages 3 and 4, 25 and 26, 27 and 28, these pairs of stages are perforated at the same time
# there are 90 perfs in total ; the first perf is nb 10 (compared to unstimulated side)
# it is the first perf. in stage 4 (stages 2 and 3 are skipped)
# last stage 28

sep = seppy.sep()

axes, data = sep.read_file(datapath+"ch4_das.H.sss.bp")
pdata = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch4_mute_mask.H.sss")
pmute = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch4_dxGnx.H.sss.bp")
pdxGnx = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch4_dzGnz.H.sss.bp")
pdzGnz = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch4_dxGnz.H.sss.bp")
pdxGnz = data.reshape(axes.n,order='F').T

pattr = np.loadtxt(datapath+"ch4_sattr.txt")


dt=axes.d[0]
dx=axes.d[1]
nt=axes.n[0]
ntr=axes.n[1]
ns=axes.n[2]

## MT inversion and time function calibration

In [ ]:
# for every shot
#    scan sigma1, sigma2>=sigma1, origin time
#    convolve GFs with the time function
#    build matrix G and invert the MT
#    find the smallest normalized functional
#    deduce the optimal origin time, sigma1, sigma2

origin_t=np.zeros(ns)
sigma1=np.zeros(ns)
sigma2=np.zeros(ns)
nfunc=999*np.ones(ns)
mt=np.zeros((ns,3))

# time shift boundaries in sec
otmin=-0.007
otmax=0.007
otnum=15

# condition number of the matrix G
cond=np.zeros(ns)

# sigma boundaries is sec
sigmin=dt/2
signum1=8
signum2=17

# full functional
nfunc_all=np.zeros((ns,otnum*signum1*signum2))

# predicted data
psyn=np.copy(pdata)

In [ ]:
spmin=0
spmax=90
offmin=50 # start at offset 150 m
offmax=101 # use only the near offsets because there is cycle skipping at farther offsets
offnum=offmax-offmin

# loop over shots
for s in range(spmin,spmax):
    
    if s%10==0:
        print("Processing shot %d" %s)
    
    # data norm
    d=np.reshape(pdata[s,offmin:offmax,:],(offnum*nt))
    dnorm=np.linalg.norm(d)
    
    # make copies of GFs
    pdxGnx_w=np.copy(pdxGnx[s,:,:])
    pdzGnz_w=np.copy(pdzGnz[s,:,:])
    pdxGnz_w=np.copy(pdxGnz[s,:,:])
    
    # full functional for the given shot
    func=np.zeros(0)
    
    # loop over sigmas
    for sig1 in np.linspace(sigmin,signum1*sigmin,signum1):
        for sig2 in np.linspace(sigmin,signum2*sigmin,signum2):
            
            if (sig2>=sig1):

                # loop over time origin
                for shift in np.linspace(otmin,otmax,otnum):

                    # build the shifted time function
                    w = wavelet(nt=nt, dt=dt, t0=shift, sig1=sig1, sig2=sig2, derivative=True)

                    # convolve GF with the time functions
                    for itr in range(ntr):
                        pdxGnx_w[itr,:]=dt*np.convolve(pdxGnx[s,itr,:],w,mode='same')
                        pdzGnz_w[itr,:]=dt*np.convolve(pdzGnz[s,itr,:],w,mode='same')
                        pdxGnz_w[itr,:]=dt*np.convolve(pdxGnz[s,itr,:],w,mode='same')


                    # form the matrix G
                    G0=np.vstack((np.reshape(pdxGnx_w,(nt*ntr)),np.reshape(pdzGnz_w,(nt*ntr)),2*np.reshape(pdxGnz_w,(nt*ntr))))
                    G=np.vstack((np.reshape(pdxGnx_w[offmin:offmax,:],(nt*offnum)),np.reshape(pdzGnz_w[offmin:offmax,:],(nt*offnum)),np.reshape(pdxGnz_w[offmin:offmax,:],(nt*offnum))))
                    G0=np.transpose(G0)
                    G=np.transpose(G)

                    # Invert the MT using the pseudo inverse of G
                    m=np.matmul(np.linalg.pinv(G),d)

                    # Compute predicted data
                    d_hat0=np.matmul(G0,m)
                    d_hat=np.matmul(G,m)

                    # Compute the normalized functional
                    f=np.linalg.norm(d_hat-d)/dnorm
                    func=np.append(func,f)

                    # check if the normalized residual is the smallest so far
                    if (f<nfunc[s]):
                        nfunc[s]=f
                        mt[s,:]=m
                        origin_t[s]=shift
                        sigma1[s]=sig1
                        sigma2[s]=sig2
                        psyn[s,:,:]=np.reshape(d_hat0,(ntr,nt))
                        cond[s]=np.linalg.cond(G)
            else:
                # loop over time origin
                for shift in np.linspace(otmin,otmax,otnum):
                    func=np.append(func,999)
                    
    nfunc_all[s] = func

In [ ]:
# save the outputs
np.save(datapath+'origin_t_s',origin_t)
np.save(datapath+'sigma1_s',sigma1)
np.save(datapath+'sigma2_s',sigma2)
np.save(datapath+'mt_s',mt)
np.save(datapath+'nfunc_s',nfunc)
np.save(datapath+'nfunc_all_s',nfunc_all)
np.save(datapath+'cond_s',cond)

In [ ]:
# load the outputs
origin_t=np.load(datapath+'origin_t_s.npy')
sigma1=np.load(datapath+'sigma1_s.npy')
sigma2=np.load(datapath+'sigma2_s.npy')
mt=np.load(datapath+'mt_s.npy')
nfunc=np.load(datapath+'nfunc_s.npy')
nfunc_all=np.load(datapath+'nfunc_all_s.npy')
cond=np.load(datapath+'cond_s.npy')

In [ ]:
sp=39
ff=nfunc_all[sp].reshape((signum1,signum2,otnum))
vmin=np.min(ff)
#vmax=np.max(ff)
vmax=1.1
plt.figure(figsize=(12,8))
for i in range(otnum-6):
    plt.subplot(3,3,i+1)
    plt.imshow(ff[:,:,i+3],cmap='coolwarm',extent=[1000*sigmin,1000*signum2*sigmin,1000*signum1*sigmin,1000*sigmin],vmin=vmin,vmax=vmax)
    #plt.colorbar(fraction=0.04)
    ot=otmin+(i+3)*(otmax-otmin)/(otnum-1)
    plt.title(r"$t_0$=%.0f ms" %(1000*ot))
    if (abs(origin_t[sp]-ot)<0.00001):
        plt.scatter(1000*sigma2[sp],1000*sigma1[sp],c='w',s=100)        
    plt.gca().invert_yaxis()
    if i>5:
        plt.xlabel(r'$\sigma_2$ (ms)')
    else:
        plt.gca().set_xticklabels([])
    if i in [0, 3, 6]:
        plt.ylabel(r'$\sigma_1$ (ms)')
    else:
        plt.gca().set_yticklabels([])

plt.tight_layout(pad=0., w_pad=0., h_pad=-12.)
# plt.savefig(figpath+'func_global_s.png',bbox_inches='tight',format='png')

In [ ]:
spmax=90
plt.figure(figsize=(12,10))
plt.subplot(2,2,1)
plt.hist(1000*sigma1[:spmax],bins=1000*np.linspace(2*sigmin,6*sigmin,5),align='left')
plt.xlabel(r'Optimal $\sigma_1$ (ms)',fontsize=12)
plt.ylabel("Shot count",fontsize=12)
plt.xticks([2,3,4,5])

plt.subplot(2,2,2)
n,bins,patches=plt.hist(1000*sigma2[:spmax],bins=1000*np.linspace(4*sigmin,(signum2+1)*sigmin,signum2-2),align='left')
plt.xlabel(r'Optimal $\sigma_2$ (ms)',fontsize=12)
plt.xticks()
plt.xticks(np.arange(4,signum2+1))
#plt.ylabel("Shot count")

plt.subplot(2,2,3)
plt.hist(1000*origin_t[:spmax],bins=1000*np.linspace(otmin,otmax,otnum),align='left')
plt.xlabel(r'Optimal time shift (ms)',fontsize=12)
plt.ylabel("Shot count",fontsize=12)


plt.subplot(2,2,4)
plt.hist(nfunc)
plt.xlabel(r"Normalized residual",fontsize=12)
plt.ylabel("Shot count",fontsize=12)

# plt.savefig(figpath+'hist_s1s2t_s.png',bbox_inches='tight',format='png')

print("Median sigma1 = %f sec" %np.median(sigma1))
print("Median sigma2 = %f sec" %np.median(sigma2))
print("Median t0 = %f sec" %np.median(origin_t))
print("minimum residual = %f" %np.min(nfunc))
print("maximum residual = %f" %np.max(nfunc))
print("mean residual = %f" %np.mean(nfunc))
print("median residual = %f" %np.median(nfunc))

In [ ]:
spmax=90

plt.figure(figsize=(12,5))
plt.subplot(1,3,1)
plt.hist(1000*mt[:spmax,0],bins=10)
plt.xlabel(r"$M_{11}$ (scaled N.m)",fontsize=12)
plt.ylabel("Shot count",fontsize=12)
plt.ylim([0, 40])

plt.subplot(1,3,2)
plt.hist(1000*mt[:spmax,1],bins=10)
plt.xlabel(r"$M_{33}$ (scaled N.m)",fontsize=12)
plt.gca().set_yticklabels([])
plt.ylim([0, 40])

plt.subplot(1,3,3)
plt.hist(1000*mt[:spmax,2],bins=10)
plt.xlabel(r"$M_{13}$ (scaled N.m)",fontsize=12)
plt.gca().set_yticklabels([])
plt.ylim([0, 40])

plt.tight_layout()

In [ ]:
%matplotlib inline
spmax=90
plt.hist(cond[:spmax],bins=10)
plt.xlabel("Condition number")
plt.ylabel("Shot count")

# plt.savefig(figpath+'condition_number_s.png',bbox_inches='tight',format='png')

print("Smallest condition number = %f" %np.min(cond))
print("Largest condition number = %f" %np.max(cond))

### Save the scalar MTs, the time function, and the full MTs (scalar MT x time function)

In [ ]:
pfull_time_func=np.zeros((3,100,nt))
for s in range(spmax):
    w0 = wavelet(nt=nt, dt=dt, t0=origin_t[s], sig1=sigma1[s], sig2=sigma2[s], derivative=True)
    w = ndimage.shift(w0,-(nt-1)/2.0+0.02/dt) # shift so the first sample corresponds to -20 ms (as the GF's)
    pfull_time_func[0,s,:]=mt[s,0]*w
    pfull_time_func[1,s,:]=mt[s,1]*w
    pfull_time_func[2,s,:]=mt[s,2]*w

In [ ]:
np.savetxt(datapath+"ch4_scaler_MTs_s.txt",mt, fmt='%.4f')
sep.write_file(datapath+"ch4_all_time_func_s.H", np.transpose(pfull_time_func), ds=np.array([dt,1,1]), os=np.array([0,0,0]), dpath=datapath)

### Display field and synthetic data (GFs convolved with the time function and weighted by the MTs)

In [ ]:
sp1=45
sp2=68
sp3=78
sp4=61

p=0.8
vmax=p*np.amax(pdata[sp1])

fig=plt.figure(figsize=(8, 8),dpi=300)
plt.subplot(2,2,1)
plt.imshow(np.transpose(pdata[sp1]),interpolation='sinc',aspect="auto",extent=[100,450,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
plt.gca().set_xticklabels([])
plt.ylabel('Time (s)')
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.title(r"Shot #%d" %sp1, fontsize=8)

plt.subplot(2,2,2)
plt.imshow(np.transpose(pdata[sp2]),interpolation='sinc',aspect="auto",extent=[100,450,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
plt.gca().set_xticklabels([])
plt.gca().set_yticklabels([])
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.title(r"Shot #%d" %sp2, fontsize=8)

plt.subplot(2,2,3)
plt.imshow(np.transpose(pdata[sp3]),interpolation='sinc',aspect="auto",extent=[100,450,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
plt.ylabel('Time (s)')
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.title(r"Shot #%d" %sp3, fontsize=8)

plt.subplot(2,2,4)
plt.imshow(np.transpose(pdata[sp4]),interpolation='sinc',aspect="auto",extent=[100,450,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
plt.gca().set_yticklabels([])
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.title(r"Shot #%d" %sp4, fontsize=8)

plt.tight_layout()

# plt.savefig(figpath+'das_examples_s.png',bbox_inches='tight',format='png')

In [ ]:
sp1=45
sp2=68
sp3=78
sp4=61
p=0.8
vmax=p*np.amax(pdata[sp1])


fig=plt.figure(figsize=(8, 8),dpi=300)
plt.subplot(2,2,1)
plt.imshow(np.transpose(psyn[sp1]),interpolation='sinc',aspect="auto",extent=[100,450,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
plt.gca().set_xticklabels([])
plt.ylabel('Time (s)')
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.title(r"Shot #%d" %sp1, fontsize=8)

plt.subplot(2,2,2)
plt.imshow(np.transpose(psyn[sp2]),interpolation='sinc',aspect="auto",extent=[100,450,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
plt.gca().set_xticklabels([])
plt.gca().set_yticklabels([])
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.title(r"Shot #%d" %sp2, fontsize=8)

plt.subplot(2,2,3)
plt.imshow(np.transpose(psyn[sp3]),interpolation='sinc',aspect="auto",extent=[100,450,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
plt.ylabel('Time (s)')
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.title(r"Shot #%d" %sp3, fontsize=8)

plt.subplot(2,2,4)
plt.imshow(np.transpose(psyn[sp4]),interpolation='sinc',aspect="auto",extent=[100,450,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
plt.gca().set_yticklabels([])
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.title(r"Shot #%d" %sp4, fontsize=8)

plt.tight_layout()

# plt.savefig(figpath+'syn_examples_s.png',bbox_inches='tight',format='png')

In [ ]:
sp1=45
sp2=68
p=0.8
vmax1=p*np.amax(pdata[sp1])
vmax2=p*np.amax(pdata[sp1])

fig=plt.figure(figsize=(8, 8),dpi=300)
plt.subplot(2,2,1)
plt.imshow(np.transpose(pdata[sp1]),interpolation='sinc',aspect="auto",extent=[100,450,0.3,0],cmap='Greys',vmin=-vmax1,vmax=vmax1)
plt.gca().set_xticklabels([])
plt.ylabel('Time (s)')
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.title(r"Shot #%d" %sp1, fontsize=8)

plt.subplot(2,2,2)
plt.imshow(np.transpose(pdata[sp2]),interpolation='sinc',aspect="auto",extent=[100,450,0.3,0],cmap='Greys',vmin=-vmax2,vmax=vmax2)
plt.gca().set_xticklabels([])
plt.gca().set_yticklabels([])
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.title(r"Shot #%d" %sp2, fontsize=8)

plt.subplot(2,2,3)
plt.imshow(np.transpose(psyn[sp1]),interpolation='sinc',aspect="auto",extent=[100,450,0.3,0],cmap='Greys',vmin=-vmax1,vmax=vmax1)
plt.xlabel('Offset (m)')
plt.ylabel('Time (s)')
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

plt.subplot(2,2,4)
plt.imshow(np.transpose(psyn[sp2]),interpolation='sinc',aspect="auto",extent=[100,450,0.3,0],cmap='Greys',vmin=-vmax2,vmax=vmax2)
plt.xlabel('Offset (m)')
plt.gca().set_yticklabels([])
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.tight_layout()

# plt.savefig(figpath+'das_syn_examples_s.png',bbox_inches='tight',format='png')